In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os
import zipfile
import pandas as pd
import numpy as np
import requests
from io import BytesIO

# -------------------------
# Create local folders
# -------------------------
os.makedirs("MetroDataset/Failure", exist_ok=True)
os.makedirs("MetroDataset/Normal", exist_ok=True)

# -------------------------
# URLs for the failure ZIPs (raw GitHub links)
# -------------------------
urls_failure = {
    'x': "https://github.com/EnfangCui/MetroDataset/raw/master/Failure/Metro_vibration_v1_x_axis_failure.zip",
    'y': "https://github.com/EnfangCui/MetroDataset/raw/master/Failure/Metro_vibration_v1_y_axis_failure.zip",
    'z': "https://github.com/EnfangCui/MetroDataset/raw/master/Failure/Metro_vibration_v1_z_axis_failure.zip"
}

urls_normal = {
    'x': "https://github.com/EnfangCui/MetroDataset/raw/master/Normal/Metro_vibration_v1_x_axis_normal.zip",
    'y': "https://github.com/EnfangCui/MetroDataset/raw/master/Normal/Metro_vibration_v1_y_axis_normal.zip",
    'z': "https://github.com/EnfangCui/MetroDataset/raw/master/Normal/Metro_vibration_v1_z_axis_normal.zip"
}

# -------------------------
# Helper function to download & unzip
# -------------------------
def download_and_unzip(url, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    r = requests.get(url)
    with zipfile.ZipFile(BytesIO(r.content)) as z:
        z.extractall(save_dir)

# Download & unzip all
for axis, url in urls_failure.items():
    download_and_unzip(url, f"MetroDataset/Failure/{axis}")

for axis, url in urls_normal.items():
    download_and_unzip(url, f"MetroDataset/Normal/{axis}")

print("All files downloaded and extracted successfully!")

# -------------------------
# Load CSVs into arrays
# -------------------------
def load_axis(base_folder):
    axes = ['x', 'y', 'z']  # ensure correct order
    arrays = []
    for axis in axes:
        folder_path = os.path.join(base_folder, axis)
        files = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv')])
        axis_data = []
        for f in files:
            df = pd.read_csv(os.path.join(folder_path, f))
            axis_data.append(df.values.flatten())
        arrays.append(np.concatenate(axis_data))  # 1D array per axis
    # Stack axes as columns
    return np.column_stack(arrays)  # shape: [samples, 3]

normal = load_axis("MetroDataset/Normal")
failure = load_axis("MetroDataset/Failure")

print("Normal shape:", normal.shape)
print("Failure shape:", failure.shape)


All files downloaded and extracted successfully!
Normal shape: (7270400, 3)
Failure shape: (12462080, 3)


In [ ]:
# -------------------------
# Save the best model
# -------------------------

# Option 1: Save entire model (architecture + weights + optimizer state)
model.save("metro_vibration_model.h5")
print("Model saved as 'metro_vibration_model.h5'")

# Option 2 (optional): Save in TensorFlow SavedModel format
model.save("metro_vibration_model_tf", save_format="tf")
print("Model saved in TensorFlow SavedModel format as 'metro_vibration_model_tf'")


In [11]:
# ===============================
# Metro Vibration Classification Pipeline (Conv1D + LSTM + Engineered Features)
# ===============================

import os
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# ------------------------------
# Step 0: Load your data
# ------------------------------
# normal and failure are already loaded
X = np.vstack([normal, failure])
y = np.hstack([np.zeros(normal.shape[0]), np.ones(failure.shape[0])])
print("Raw X shape:", X.shape, "y shape:", y.shape)

# ------------------------------
# Step 1: Sliding window with engineered features
# ------------------------------
def extract_features(window):
    """
    Extract basic features per axis for a window
    Returns shape: [window_size, 3 + 3] -> original + engineered features
    """
    # original signals
    x = window[:, 0]
    y_ = window[:, 1]
    z = window[:, 2]
    
    # engineered features per axis: mean, std, RMS
    feats = np.stack([
        np.mean(x), np.std(x), np.sqrt(np.mean(x**2)),
        np.mean(y_), np.std(y_), np.sqrt(np.mean(y_**2)),
        np.mean(z), np.std(z), np.sqrt(np.mean(z**2))
    ])
    # repeat across time dimension to concatenate
    feats_tile = np.tile(feats, (window.shape[0], 1))  # [window_size, 9]
    return np.hstack([window, feats_tile])  # [window_size, 3+9=12]

def create_windows_features(X, y, window_size=500, step=200, threshold=0.3):
    X_windows = []
    y_windows = []
    for i in range(0, len(X) - window_size + 1, step):
        window = X[i:i+window_size]
        X_windows.append(extract_features(window))
        if y[i:i+window_size].mean() > threshold:
            y_windows.append(1)
        else:
            y_windows.append(0)
    return np.array(X_windows), np.array(y_windows)

window_size = 500
step_size = 200
threshold = 0.3

X_windows, y_windows = create_windows_features(X, y, window_size, step_size, threshold)
print("Windows with features shape:", X_windows.shape, "Labels shape:", y_windows.shape)

# ------------------------------
# Step 2: Scale features
# ------------------------------
num_windows, ws, num_features = X_windows.shape
X_flat = X_windows.reshape(num_windows*ws, num_features)

scaler = StandardScaler()
X_flat_scaled = scaler.fit_transform(X_flat)
X_windows_scaled = X_flat_scaled.reshape(num_windows, ws, num_features)

joblib.dump(scaler, "scaler.save")
print("Scaler saved as 'scaler.save'")

# ------------------------------
# Step 3: Train-test split
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_windows_scaled, y_windows, test_size=0.2, stratify=y_windows, random_state=42
)
print("X_train:", X_train.shape, "X_test:", X_test.shape)

# ------------------------------
# Step 4: Class weights
# ------------------------------
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weights_dict = dict(zip(classes, class_weights))
print("Class weights:", class_weights_dict)

# ------------------------------
# Step 5: Conv1D + LSTM model
# ------------------------------
model = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(window_size, num_features)),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.LSTM(64, return_sequences=False),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ------------------------------
# Step 6: Train model
# ------------------------------
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = callbacks.ModelCheckpoint("best_metro_cnn_lstm.keras", save_best_only=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=512,
    class_weight=class_weights_dict,
    callbacks=[early_stop, checkpoint],
    verbose=2
)

# ------------------------------
# Step 7: Evaluate model
# ------------------------------
y_pred_prob = model.predict(X_test)
# tune threshold for better F1
best_thresh = 0.4
y_pred = (y_pred_prob > best_thresh).astype(int)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

# ------------------------------
# Step 8: Multi-agent CSV inference
# ------------------------------
def predict_csv(model, csv_path, scaler_path="scaler.save", window_size=500, step_size=200, threshold=0.3, best_thresh=0.4):
    scaler = joblib.load(scaler_path)
    df = pd.read_csv(csv_path)
    X = df[['x','y','z']].values
    
    # create windows
    X_windows, _ = create_windows_features(X, np.zeros(len(X)), window_size, step_size, threshold)
    
    # scale
    num_windows, ws, num_features = X_windows.shape
    X_flat = X_windows.reshape(num_windows*ws, num_features)
    X_scaled = scaler.transform(X_flat).reshape(num_windows, ws, num_features)
    
    # predict
    preds_prob = model.predict(X_scaled)
    preds = (preds_prob > best_thresh).astype(int)
    return preds

# Example usage:
# model = tf.keras.models.load_model("best_metro_cnn_lstm.keras")
# preds = predict_csv(model, "agent_input.csv")
# print("Predictions per window:", preds)


Raw X shape: (19732480, 3) y shape: (19732480,)
Windows with features shape: (98660, 500, 12) Labels shape: (98660,)
Scaler saved as 'scaler.save'
X_train: (78928, 500, 12) X_test: (19732, 500, 12)
Class weights: {0: 1.357037240810151, 1: 0.7917026099865588}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_9 (Conv1D)               │ (None, 498, 32)        │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 496, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 248, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,641 (174.38 KB)

 Trainable params: 44,641 (174.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
124/124 - 10s - 81ms/step - accuracy: 0.6274 - loss: 0.5395 - val_accuracy: 0.5869 - val_loss: 0.5810
Epoch 2/20
124/124 - 5s - 38ms/step - accuracy: 0.6206 - loss: 0.5421 - val_accuracy: 0.6373 - val_loss: 0.5456
Epoch 3/20
124/124 - 5s - 38ms/step - accuracy: 0.6222 - loss: 0.5337 - val_accuracy: 0.6358 - val_loss: 0.5356
Epoch 4/20
124/124 - 5s - 38ms/step - accuracy: 0.6249 - loss: 0.5288 - val_accuracy: 0.5926 - val_loss: 0.5619
Epoch 5/20
124/124 - 5s - 38ms/step - accuracy: 0.6131 - loss: 0.5389 - val_accuracy: 0.6340 - val_loss: 0.5428
Epoch 6/20
124/124 - 5s - 38ms/step - accuracy: 0.6312 - loss: 0.5247 - val_accuracy: 0.6304 - val_loss: 0.5356
Epoch 7/20
124/124 - 5s - 38ms/step - accuracy: 0.6210 - loss: 0.5316 - val_accuracy: 0.5825 - val_loss: 0.5680
Epoch 8/20
124/124 - 5s - 38ms/step - accuracy: 0.6289 - loss: 0.5263 - val_accuracy: 0.6361 - val_loss: 0.5330
Epoch 9/20
124/124 - 5s - 38ms/step - accuracy: 0.6312 - loss: 0.5235 - val_accuracy: 0.6356 - val_loss

In [12]:
import pandas as pd

# mean/std per window
means = X_windows.mean(axis=1)  # shape: [num_windows, num_features]
stds = X_windows.std(axis=1)
df_summary = pd.DataFrame(np.hstack([means, stds]), columns=[f"mean_{i}" for i in range(X_windows.shape[2])] + [f"std_{i}" for i in range(X_windows.shape[2])])
df_summary['label'] = y_windows
df_summary.groupby('label').describe()


mean_0                                                     \
         count          mean            std      min          25%   
label                                                               
0      36351.0  9.088023e+05  524674.847865  62.2696  454437.2613   
1      62309.0  1.557724e+06  899344.669360  62.2758  778862.2830   

                                                  mean_1                ...  \
                50%           75%           max    count          mean  ...   
label                                                                   ...   
0      9.088123e+05  1.363187e+06  1.817512e+06  36351.0  9.088023e+05  ...   
1      1.557712e+06  2.336562e+06  3.115412e+06  62309.0  1.557724e+06  ...   

             std_10                 std_11                                   \
                75%           max    count          mean           std  min   
label                                                                         
0      8.149073e-09  2.421439e-08  36351.0  7.548222e-09  6.499806e-09  0.0   
1      1.350418e-08  3.911555e-08  62309.0  1.370600e-08  1.193840e-08  0.0   

                                                               
                25%           50%           75%           max  
label                                                          
0      2.240995e-09  5.820766e-09  1.140870e-08  3.445894e-08  
1      3.841706e-09  1.059379e-08  2.072193e-08  6.146729e-08  

[2 rows x 192 columns]

In [21]:
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import xgboost as xgb
from scipy.stats import skew, kurtosis

# -------------------------
# Parameters
# -------------------------
WINDOW_SIZE = 100      # timesteps per window
MAX_WINDOWS = 50000    # total windows (split equally between Normal & Failure)
PRINT_EVERY = 1000
RANDOM_STATE = 42

# -------------------------
# Load CSVs
# -------------------------
def load_axis_data(folder):
    all_data = []
    for axis in ['x','y','z']:
        axis_path = os.path.join(folder, axis)
        files = sorted(os.listdir(axis_path))
        axis_values = []
        for f in files:
            df = pd.read_csv(os.path.join(axis_path, f))
            axis_values.append(df.values.flatten())
        axis_array = np.concatenate(axis_values)
        all_data.append(axis_array)
    return np.vstack(all_data).T  # shape [samples, 3]

print("Loading Normal data...")
normal_data = load_axis_data("MetroDataset/Normal")
print("Loading Failure data...")
failure_data = load_axis_data("MetroDataset/Failure")

print("Normal shape:", normal_data.shape, "Failure shape:", failure_data.shape)

# -------------------------
# Windowing and feature extraction
# -------------------------
def extract_features(data, num_windows, window_size, print_every=1000):
    X_windows = []
    for i in range(1, num_windows + 1):
        start = np.random.randint(0, data.shape[0] - window_size)
        window = data[start:start+window_size, :]
        features = []
        for axis in range(window.shape[1]):  # x, y, z
            axis_data = window[:, axis]
            features.extend([
                axis_data.mean(),
                axis_data.std(),
                axis_data.min(),
                axis_data.max(),
                skew(axis_data),
                kurtosis(axis_data),
                np.median(axis_data),
                np.percentile(axis_data, 25),
                np.percentile(axis_data, 75),
                np.ptp(axis_data),                 # max-min
                np.sum(np.abs(np.diff(axis_data))), # roughness
                np.mean(np.abs(np.diff(axis_data))) # mean diff
            ])
        X_windows.append(features)

        if i % print_every == 0:
            print(f"Processed {i} windows...")

    return np.array(X_windows)

num_windows_per_class = MAX_WINDOWS // 2

print("Extracting Normal windows...")
X_normal = extract_features(normal_data, num_windows_per_class, WINDOW_SIZE, PRINT_EVERY)
y_normal = np.zeros(num_windows_per_class)

print("Extracting Failure windows...")
X_failure = extract_features(failure_data, num_windows_per_class, WINDOW_SIZE, PRINT_EVERY)
y_failure = np.ones(num_windows_per_class)

# -------------------------
# Combine and shuffle
# -------------------------
X_features = np.vstack([X_normal, X_failure])
y_windows = np.hstack([y_normal, y_failure])

X_features, y_windows = shuffle(X_features, y_windows, random_state=RANDOM_STATE)

print("Feature matrix shape:", X_features.shape)
print("Labels shape:", y_windows.shape)

# -------------------------
# Train-test split
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_windows, test_size=0.2, stratify=y_windows, random_state=RANDOM_STATE
)

# Standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -------------------------
# Train XGBoost
# -------------------------
num_class0 = np.sum(y_train==0)
num_class1 = np.sum(y_train==1)
scale_pos_weight = num_class0 / num_class1

clf = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=RANDOM_STATE
)

print("Training XGBoost model...")
clf.fit(X_train_scaled, y_train)

# -------------------------
# Evaluate
# -------------------------
y_pred = clf.predict(X_test_scaled)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


Loading Normal data...
Loading Failure data...
Normal shape: (7270400, 3) Failure shape: (12462080, 3)
Extracting Normal windows...
Processed 1000 windows...
Processed 2000 windows...
Processed 3000 windows...
Processed 4000 windows...
Processed 5000 windows...
Processed 6000 windows...
Processed 7000 windows...
Processed 8000 windows...
Processed 9000 windows...
Processed 10000 windows...
Processed 11000 windows...
Processed 12000 windows...
Processed 13000 windows...
Processed 14000 windows...
Processed 15000 windows...
Processed 16000 windows...
Processed 17000 windows...
Processed 18000 windows...
Processed 19000 windows...
Processed 20000 windows...
Processed 21000 windows...
Processed 22000 windows...
Processed 23000 windows...
Processed 24000 windows...
Processed 25000 windows...
Extracting Failure windows...
Processed 1000 windows...
Processed 2000 windows...
Processed 3000 windows...
Processed 4000 windows...
Processed 5000 windows...
Processed 6000 windows...
Processed 7000 w

In [22]:
import joblib

# Save the trained XGBoost model
model_path = "xgb_metro_model.pkl"
joblib.dump(clf, model_path)
print(f"Model saved to {model_path}")

# Save the StandardScaler too (for scaling new input)
scaler_path = "scaler_metro.pkl"
joblib.dump(scaler, scaler_path)
print(f"Scaler saved to {scaler_path}")


Model saved to xgb_metro_model.pkl
Scaler saved to scaler_metro.pkl
